<a href="https://colab.research.google.com/github/fabiobaroliveira/gerador_base_de_dados/blob/main/locadora_de_ve%C3%ADculos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Criando base de dados de uma locadora de veículos

---
- Clientes
- Locações
- Frota de veículos


In [1]:
#Necessário instalar a biblioteca faker
! pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 23.0 MB/s eta 0:00:00


#Definido alguns parametros

In [2]:
import pandas as pd
from faker import Faker
import random
from datetime import datetime, timedelta
fake = Faker('pt_BR')

# Configurações
NUM_CLIENTES = 100  # Número de clientes
NUM_VEICULOS = 50   # Número de veículos
NUM_LOCACOES = 20000 # Número de locações

# Lista de veiculos
modelos_categorias = [
    {"marca": "Fiat", "modelo": "Mobi", "categoria": "Econômico"},
    {"marca": "Fiat", "modelo": "Argo", "categoria": "Econômico"},
    {"marca": "Fiat", "modelo": "Cronos", "categoria": "Intermediário"},
    {"marca": "Fiat", "modelo": "Pulse", "categoria": "SUV"},
    {"marca": "Fiat", "modelo": "Toro", "categoria": "Pickup"},
    {"marca": "Volkswagen", "modelo": "Gol", "categoria": "Econômico"},
    {"marca": "Volkswagen", "modelo": "Polo", "categoria": "Intermediário"},
    {"marca": "Volkswagen", "modelo": "Virtus", "categoria": "Intermediário"},
    {"marca": "Volkswagen", "modelo": "T-Cross", "categoria": "SUV"},
    {"marca": "Chevrolet", "modelo": "Onix", "categoria": "Econômico"},
    {"marca": "Chevrolet", "modelo": "Cruze", "categoria": "Intermediário"},
    {"marca": "Chevrolet", "modelo": "Tracker", "categoria": "SUV"},
    {"marca": "Chevrolet", "modelo": "S10", "categoria": "Pickup"},
    {"marca": "Hyundai", "modelo": "HB20", "categoria": "Econômico"},
    {"marca": "Hyundai", "modelo": "Creta", "categoria": "SUV"},
    {"marca": "Hyundai", "modelo": "Tucson", "categoria": "SUV"},
    {"marca": "Toyota", "modelo": "Corolla", "categoria": "Intermediário"},
    {"marca": "Toyota", "modelo": "Hilux", "categoria": "Pickup"},
    {"marca": "Toyota", "modelo": "SW4", "categoria": "SUV"},
    {"marca": "Ford", "modelo": "Ka", "categoria": "Econômico"},
    {"marca": "Ford", "modelo": "EcoSport", "categoria": "SUV"},
    {"marca": "Ford", "modelo": "Ranger", "categoria": "Pickup"}
]

# Listas de opções

cores = ["Preto", "Branco", "Prata", "Vermelho", "Azul", "Cinza"]

status_locacao = ["Ativa", "Finalizada", "Cancelada"]

cidades_estados = { "São Paulo": "SP", "Rio de Janeiro": "RJ","Belo Horizonte": "MG",
    "Porto Alegre": "RS","Curitiba": "PR",  "Salvador": "BA", "Fortaleza": "CE",
    "Recife": "PE", "Brasília": "DF", "Manaus": "AM","Belém": "PA","Goiânia": "GO",
    "Campinas": "SP", "São Luís": "MA","Maceió": "AL","Natal": "RN","Teresina": "PI",
    "João Pessoa": "PB","Aracaju": "SE", "Florianópolis": "SC","Vitória": "ES",
    "Cuiabá": "MT", "Campo Grande": "MS","Porto Velho": "RO","Boa Vista": "RR",
    "Palmas": "TO","Rio Branco": "AC","Macapá": "AP"
}

# Dicionário de faixas de preço por categoria
faixas_preco = {
    "Econômico": (100, 170),
    "Intermediário": (180, 250),
    "SUV": (350, 420),
    "Pickup": (300, 400)
}



# Gerar veículos

In [3]:
# Gerar veículos
veiculos = []
for i in range(1, NUM_VEICULOS + 1):
    veiculo = random.choice(modelos_categorias)  # Escolhe um veículo aleatório da lista
    ano = random.randint(2015, 2023)
    placa = f"{random.choice(['ABC', 'DEF', 'GHI'])}{random.randint(1000, 9999)}"
    veiculos.append({
        "id_veiculo": i,
        "marca": veiculo["marca"],
        "modelo": veiculo["modelo"],
        "ano": ano,
        "placa": placa,
        "categoria": veiculo["categoria"],
        "cor": random.choice(cores),
        "quilometragem": random.randint(0, 100000),
        "disponivel": random.choice([True, False])
    })

df_veiculos = pd.DataFrame(veiculos)

# Gerar lojas

In [4]:
# Gerar lojas
lojas = []
for i, (cidade, estado) in enumerate(cidades_estados.items(), start=1):
    lojas.append({
        "id_loja": i,
        "nome_loja": f"Locadora {cidade}",
        "cidade": cidade,
        "estado": estado
    })

df_lojas = pd.DataFrame(lojas)

# Gerar clientes

In [5]:
# Gerar clientes
clientes = []
for i in range(1, NUM_CLIENTES + 1):
    sexo = random.choice(["Masculino", "Feminino"])
    nome = fake.first_name_male() if sexo == "Masculino" else fake.first_name_female()
    cidade, estado = random.choice(list(cidades_estados.items()))  # Escolhe uma cidade e estado aleatórios
    clientes.append({
        "id_cliente": i,
        "nome": f"{nome} {fake.last_name()}",
        "cpf": fake.cpf(),
        "email": fake.email(),
        "telefone": fake.phone_number(),
        "data_nascimento": fake.date_of_birth(minimum_age=18, maximum_age=70),
        "sexo": sexo,
        "cidade": cidade,
        "estado": estado
    })

df_clientes = pd.DataFrame(clientes)

#Gerar locações

In [6]:
# Gerar locações

# Função para gerar datas aleatórias
def gerar_data_aleatoria(inicio, fim):
    return inicio + timedelta(days=random.randint(0, (fim - inicio).days))

# Função para gerar o valor da diária com base na categoria
def gerar_valor_diaria(categoria):
    if categoria in faixas_preco:
        min_valor, max_valor = faixas_preco[categoria]
        return round(random.uniform(min_valor, max_valor), 2)

locacoes = []
data_inicio = datetime(2022, 1, 1)
data_fim = datetime(2025, 12, 31)

for i in range(1, NUM_LOCACOES + 1):
    id_cliente = random.randint(1, NUM_CLIENTES)
    id_veiculo = random.randint(1, NUM_VEICULOS)
    # Recupera a categoria do veículo
    veiculo = df_veiculos[df_veiculos['id_veiculo'] == id_veiculo].iloc[0]
    categoria = veiculo['categoria']
    # Gera o valor da diária com base na categoria
    valor_diaria = gerar_valor_diaria(categoria)
    data_locacao = gerar_data_aleatoria(data_inicio, data_fim)
    data_devolucao = data_locacao + timedelta(days=random.randint(1, 30))
    dias_locacao = (data_devolucao - data_locacao).days
    valor_total = round(valor_diaria * dias_locacao, 2)
    status = random.choice(status_locacao)

# Recupera a cidade do cliente
    cliente = df_clientes[df_clientes['id_cliente'] == id_cliente].iloc[0]
    cidade_cliente = cliente['cidade']

# Encontra a loja correspondente à cidade do cliente
    loja = df_lojas[df_lojas['cidade'] == cidade_cliente].iloc[0]
    id_loja = loja['id_loja']


    locacoes.append({
        "id_locacao": i,
        "id_loja": id_loja,
        "id_cliente": id_cliente,
        "id_veiculo": id_veiculo,
        "data_locacao": data_locacao.strftime('%d-%m-%Y'),
        "data_devolucao": data_devolucao.strftime('%d-%m-%Y'),
        "valor_diaria": valor_diaria,
        "dias_locacao": dias_locacao,
        "valor_total": valor_total,
        "status": status

    })

df_locacoes = pd.DataFrame(locacoes)

#Substituindo . dos valores e ajustando ao padrão brasileiro
df_locacoes['valor_diaria'] = df_locacoes['valor_diaria'].astype(str).str.replace('.', ',')
df_locacoes['valor_total'] = df_locacoes['valor_total'].astype(str).str.replace('.', ',')

# Exibir DataFrames

In [7]:
print("\nVeículos:")
df_veiculos.head()


Veículos:


,id_veiculo,marca,modelo,ano,placa,categoria,cor,quilometragem,disponivel
0,1,Ford,Ranger,2018,ABC4032,Pickup,Preto,30704,False
1,2,Toyota,Corolla,2023,GHI6003,Intermediário,Vermelho,76290,True
2,3,Fiat,Cronos,2015,ABC8494,Intermediário,Branco,47717,True
3,4,Hyundai,Creta,2017,ABC6550,SUV,Prata,71142,True
4,5,Fiat,Argo,2023,ABC9833,Econômico,Azul,67326,False


In [8]:
print("Lojas:")
df_lojas.head()

Lojas:


,id_loja,nome_loja,cidade,estado
0,1,Locadora São Paulo,São Paulo,SP
1,2,Locadora Rio de Janeiro,Rio de Janeiro,RJ
2,3,Locadora Belo Horizonte,Belo Horizonte,MG
3,4,Locadora Porto Alegre,Porto Alegre,RS
4,5,Locadora Curitiba,Curitiba,PR


In [9]:
print("Clientes:")
df_clientes.head()

Clientes:


,id_cliente,nome,cpf,email,telefone,data_nascimento,sexo,cidade,estado
0,1,Ágatha Aragão,152.768.049-58,joao-pedrosantos@example.org,(031) 9600 7691,1975-05-16,Feminino,Goiânia,GO
1,2,Yuri Cavalcanti,860.239.514-06,theonogueira@example.com,0300-584-9410,1958-09-23,Masculino,Goiânia,GO
2,3,Thomas da Rosa,571.930.842-32,daniel94@example.net,+55 71 7365-3237,2002-04-22,Masculino,Recife,PE
3,4,Vitória Sales,247.381.096-69,jade97@example.com,(061) 2230 3989,1968-02-02,Feminino,Goiânia,GO
4,5,Gustavo da Rosa,408.761.325-90,alice82@example.net,+55 21 7182 6202,2006-04-20,Masculino,Maceió,AL


In [10]:
print("\nLocações:")
df_locacoes.head()


Locações:


,id_locacao,id_loja,id_cliente,id_veiculo,data_locacao,data_devolucao,valor_diaria,dias_locacao,valor_total,status
0,1,14,94,40,16-03-2024,01-04-2024,"108,29",16,"1732,64",Cancelada
1,2,22,80,27,21-06-2024,24-06-2024,"375,44",3,"1126,32",Ativa
2,3,26,71,4,31-12-2025,24-01-2026,"401,15",24,"9627,6",Ativa
3,4,11,59,15,12-03-2025,28-03-2025,"212,27",16,"3396,32",Finalizada
4,5,18,7,15,22-09-2024,04-10-2024,"227,24",12,"2726,88",Ativa


#Criando os arquivos .csv

In [11]:
from google.colab import files

# Cria arquivos .csv com o separador ;
df_veiculos.to_csv('veiculos.csv', index=False, sep=';')
df_lojas.to_csv('lojas.csv', index=False, sep=';')
df_clientes.to_csv('clientes.csv', index=False, sep=';')
df_locacoes.to_csv('locacoes.csv', index=False, sep=';')

# Faz o download dos arquivos
files.download('veiculos.csv')
files.download('lojas.csv')
files.download('clientes.csv')
files.download('locacoes.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>